In [1]:
import torch
import torch.nn as nn

pytorch 사용.
nn으로 줄임.

In [3]:
x = torch.tensor([[1,1,0],[4,2,1],[0,2,1]])
print('ndim =', x.ndim)
print(x.shape)
print(x)
y = torch.flatten(x)
print('ndim =', y.ndim)
print(y.shape)
print(y)

ndim = 2
torch.Size([3, 3])
tensor([[1, 1, 0],
        [4, 2, 1],
        [0, 2, 1]])
ndim = 1
torch.Size([9])
tensor([1, 1, 0, 4, 2, 1, 0, 2, 1])


x.ndim : x텐서의 차원 수 반환하는 속성
x.shape: 텐서의 각 차원 크기 반환하는 속성. 
x: 텐서 전체 출력
torch.flatten: 1차원으로 평탄화
y: 평탄화된 x


In [5]:
x = torch.tensor([[[1,1,0],[4,2,1],[0,2,1]]])
print('ndim =', x.ndim)
print(x.shape)
print(x)
y = torch.flatten(x,1)
print('ndim =', y.ndim)
print(y.shape)
print(y)

ndim = 3
torch.Size([1, 3, 3])
tensor([[[1, 1, 0],
         [4, 2, 1],
         [0, 2, 1]]])
ndim = 2
torch.Size([1, 9])
tensor([[1, 1, 0, 4, 2, 1, 0, 2, 1]])


x.ndim : x텐서의 차원 수 반환하는 속성; 차원수 3
x.shape: 텐서의 각 차원 크기 반환하는 속성. 
x: 텐서 전체 출력
torch.flatten: 2차원으로 평탄화, 단 첫 번째는 놔두고 두 번째 차원부터 끝까지.
y: 평탄화된 x

In [7]:
x = torch.tensor([[[1,1,0],[4,2,1],[0,2,1]]])
print('ndim =', x.ndim)
print(x.shape)
model = nn.Sequential(
    nn.Flatten()
)
print(model)
y = model(x)
print('ndim =', y.ndim)
print(y.shape)

ndim = 3
torch.Size([1, 3, 3])
Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
)
ndim = 2
torch.Size([1, 9])


nn.Sequential() : 레이어 순차적으로 쌓는 컨테이너
-> 이 코드에선 Flatten 레이어 하나만 통과.
 - start_dim=1: 평탄화 시작 차원(1번 차원부터.0번 차원(배치)는 유지.)
 - end_dim=-1: 평탄화 끝낼 차원(마지막 차원).

CNN 모델 파이토치 구현

In [10]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.utils as utils
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device:', device)

Device: cpu


numpy, matplotlib, torch, torchvision 등 딥러닝 데이터 시각화에 필요한 라이브러리 임포트.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
-> cuda(GPU) 설치됐다면 cuda, 아니면 CPU.

print('Device:', device)
-> 현재 device는 CPU.

이렇게 지정한 device는 모델이나 데이터를 해당 device로 옮길 때 사용.




In [12]:
train_dataset = torchvision.datasets.MNIST(root='MNIST_data', 
                                        train=True,
                                        download=True,
                                        transform=None)

test_dataset = torchvision.datasets.MNIST(root='MNIST_data', 
                                        train=False, 
                                        download=True, 
                                        transform=None)

train_dataset = torchvision.datasets.MNIST(root='MNIST_data', 
                                        train=True,
                                        download=True,
                                        transform=None)
                                        
- root='MNIST_data': 데이터셋 저장될 경로 지정. 
- train=True: 학습용 데이터셋(훈련데이터) 불러오기.
- download=True: 해당 경로에 데이터 없으면 자동으로 다운로드.
- transform=None: 데이터에 적용할 변환(transform)을 지정. None이면 변환 없이 원본 데이터 사용.

test_dataset = torchvision.datasets.MNIST(root='MNIST_data', 
                                        train=False, 
                                        download=True, 
                                        transform=None)

- root='MNIST_data': 데이터셋 저장될 경로 지정. 
- train=False: 테스용 데이터셋(검증 데이터) 불러오기.
- download=True: 해당 경로에 데이터 없으면 자동으로 다운로드.
- transform=None: 데이터에 적용할 변환(transform)을 지정. None이면 변환 없이 원본 데이터 사용.

In [14]:
print(train_dataset)
print(test_dataset)

Dataset MNIST
    Number of datapoints: 60000
    Root location: MNIST_data
    Split: Train
Dataset MNIST
    Number of datapoints: 10000
    Root location: MNIST_data
    Split: Test


MNIST 데이터셋은 손글씨 숫자 이미지(0~9)와 해당 숫자 레이블로 구성되어 있으며, 딥러닝 모델 학습에 자주 사용되는 대표적인 이미지 분류 데이터셋.

학습용 : 60000개
테스트용 : 10000개
로 데이터분할. 

In [16]:
print(type(train_dataset[0]))
print(type(test_dataset[0]))
print(train_dataset[0])
print(test_dataset[0])

<class 'tuple'>
<class 'tuple'>
(<PIL.Image.Image image mode=L size=28x28 at 0x1DFC6CA2C00>, 5)
(<PIL.Image.Image image mode=L size=28x28 at 0x1DFC2B05880>, 7)


print(type(train_dataset[0]))
print(type(test_dataset[0]))
학습과 테스트 데이터셋에서 첫 번째 샘플 가져와 타입인 튜플 출력.
print(train_dataset[0])
print(test_dataset[0])
각 튜플의 첫 번째 원소는 PIL 이미지 객체(mode='L', 즉 흑백 28*28 이미지)
두 번째 원소는 정수형 레이블(0~9의 숫자)

In [ ]:
plt.imshow(train_dataset[0][0], cmap='gray')
plt.title('%i' % train_dataset[0][1])
plt.show()

plt.imshow(train_dataset[0][0], cmap='gray')

- train_dataset은 (이미지, 레이블) 형태의 튜플.
- 따라서 train_dataset[0][0]은 첫 번째 train_dataset의 PIL.Image.Image 객체.
- cmap = 'gray' 이미지를 흑백으로 시각화.
  
plt.title('%i' % train_dataset[0][1])

- 제목을 해당 데이터의 튜플 속 레이블로.
  
plt.show()
- 화면 출력. 

In [ ]:
#numpy 배열로 변환
print(type(train_dataset[0][0]))
image = np.array(train_dataset[0][0])
print(type(image))
print(image)

print(type(train_dataset[0][0])) 
- train_dataset[0][0]은 첫 번째 train_dataset의 PIL.Image.Image 객체이므로
 class 'PIL.Image.Image' 출력.
 
image = np.array(train_dataset[0][0])
- PIL 이미지를 numpy 배열로 변환.

print(type(image))
- <class 'numpy.ndarray'> numpy 배열로 변환됐음을 확인.
  
print(image)
- 28x28 크기의 2차원 numpy 배열이 출력. 각 원소는 픽셀의 밝기(0~255) 값.

In [ ]:
#MNIST 데이터를 텐서로 변환
transform = transforms.ToTensor()     

train_dataset = torchvision.datasets.MNIST(root='MNIST_data', 
                                        train=True,
                                        download=True,
                                        transform=transform)

test_dataset = torchvision.datasets.MNIST(root='MNIST_data', 
                                        train=False, 
                                        download=True, 
                                        transform=transform)

print(train_dataset)
print(test_dataset)

transform = transforms.ToTensor()
- 이미지를 0~1 실수값의 torch.FloatTensor로 변환.

print(train_dataset)
print(test_dataset)
- 데이터셋 객체 정보 출력.

- root='MNIST_data': 데이터셋 저장될 경로 지정. 
- train=True: 학습용 데이터셋(훈련데이터) 불러오기.
- train=False: 테스용 데이터셋(검증 데이터) 불러오기.
- download=True: 해당 경로에 데이터 없으면 자동으로 다운로드.
- transform=transform: 데이터에 적용할 변환(transform)을 지정. PIL 이미지 -> torch.Tensor로 자동 변환해서 반환.

In [ ]:
def imshow(img):
#    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)), cmap='gray' )
    plt.show()

<plt.imshow() 구현>

npimg = img.numpy()
- PIL이미지인 img -> numpy배열인 npimg로 변환.

plt.imshow(np.transpose(npimg, (1, 2, 0)), cmap='gray' )
- PyTorch 이미지 텐서는 (채널, 높이, 너비)순.
- matplotlib는 (높이, 너비, 채널)순서를 기대하므로, np.transpose로 (0,1,2)에서 (1,2,0)으로 차원 순서 바꿔주기.
- cmap='gray'는 흑백 이미지를 그레이스케일로 표시.

plt.show()
- 화면 출력.

In [ ]:
plt.title('%i' % train_dataset[0][1])
imshow(train_dataset[0][0])

plt.title('%i' % train_dataset[0][1])

- 제목을 해당 데이터의 튜플 속 레이블로.

imshow(train_dataset[0][0])
- plt.imshow구현 확인.

In [ ]:
print(train_dataset[0][0])

- 출력 결과는 transform=transforms.ToTensor()로 인해 torch.Tensor 객체.
- shape: (1,28,28)
- type: torch.float32
- 값은 0~1 사이 실수로 이뤄진 3차원 텐서. 값이 0~255범위 정수로 유지되는 numpy 배열과는 다르다.

In [ ]:
#Compose함수로 데이터 변환 여러 가지를 묶어줌
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5),(0.5))])

train_dataset = torchvision.datasets.MNIST(root='MNIST_data', 
                                        train=True,
                                        download=True,
                                        transform=transform)

test_dataset = torchvision.datasets.MNIST(root='MNIST_data', 
                                        train=False, 
                                        download=True, 
                                        transform=transform)
print('number of training data : ', len(train_dataset))
print('number of test data : ', len(test_dataset))

#Compose함수로 데이터 변환 여러 가지를 묶어줌

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5),(0.5))])

transforms.ToTensor()
PIL 이미지 -> PyTorch 텐서로 변환, 픽셀값 0~1의 실수로 자동 정규화.

transforms.Normalize((0.5),(0.5))
평균, 표준편차는 0.5라는 의미.
각 채널별로 (x-0.5)/0.5 연산 수행. 픽셀값 -1~1 범위로 변환.

나머지는 기존과 같이 데이터셋 세팅.

print('number of training data : ', len(train_dataset))
print('number of test data : ', len(test_dataset))
학습 데이터와 검증 데이터 수 출력.


In [ ]:
print(train_dataset)
print(test_dataset)

학습 데이터와 검증 데이터 특성 출력.

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)), cmap='gray' )
    plt.show()

<plt.imshow() 구현>

npimg = img.numpy()
- PIL이미지인 img -> numpy배열인 npimg로 변환.

plt.imshow(np.transpose(npimg, (1, 2, 0)), cmap='gray' )
- PyTorch 이미지 텐서는 (채널, 높이, 너비)순.
- matplotlib는 (높이, 너비, 채널)순서를 기대하므로, np.transpose로 (0,1,2)에서 (1,2,0)으로 차원 순서 바꿔주기.
- cmap='gray'는 흑백 이미지를 그레이스케일로 표시.

plt.show()
- 화면 출력.

In [ ]:
imshow(train_dataset[0][0])

imshow(train_dataset[0][0])
- plt.imshow구현 확인.

In [ ]:
print(train_dataset[0][0])

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5),(0.5))])

transforms.ToTensor()
PIL 이미지 -> PyTorch 텐서로 변환, 픽셀값 0~1의 실수로 자동 정규화.

transforms.Normalize((0.5),(0.5))
평균, 표준편차는 0.5라는 의미.
각 채널별로 (x-0.5)/0.5 연산 수행. 픽셀값 -1~1 범위로 변환.

이전의 Compose내 transform이 수행된 train_dataset[0][0] 출력.

torch.Tensor 타입이고, 픽셀값은 -1~1이다.

In [ ]:
torch.manual_seed(1)
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                            batch_size=64, 
                                            shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=64, 
                                           shuffle=False) 

torch.manual_seed(1)
랜덤 시드 고정. -> 데이터 섞기에서 결과가 항상 동일.

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                            batch_size=64, 
                                            shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=64, 
                                           shuffle=False) 
                                           
- 변환이 수행된 학습데이터와 검증데이터를 불러오는 각각의 DataLoader 생성.
- batch_size=64: 한 번에 64개 샘플씩 가져옴.
- shuffle=True: epoch마다 데이터를 무작위로 섞는다.(학습 성능 향상에 도움)
- shuffle=False: 테스트 데이터는 섞지 않고 순서대로 평가.



In [ ]:
dataiter = iter(train_loader)
images, labels = next(dataiter)

print(images.shape)
print(labels.shape)

dataiter = iter(train_loader)
이터레이터 생성

images, labels = next(dataiter)
첫 번째 배치 꺼내기. 


64: 배치 크기 
1: 채널 수
28 : 이미지의 높이와 너비

64: 배치 크기 

In [ ]:
print(images[0].squeeze().shape)

squeeze()로 첫 번째 batch의 텐서의 shape에서 크기 1인 차원(채널 차원)제거 후 남은 차원 출력.

In [ ]:
print(images[0][0].shape) #위와 같은 결과

첫 번째 batch의 첫 번째 이미지 채널만 선택해 차원 크기 출력. 위와 같다.

In [ ]:
plt.imshow(images[0].squeeze(), cmap='gray')

plt.imshow(images[0].squeeze(), cmap='gray')

- 현재 배치의 첫 번째 이미지를 squeeze()로 차원 축소 후, Matplotlib를 사용해 흑백(cmap='gray')으로 표시. 이 이미지는 Normalize((0.5), (0.5)) 변환이 적용되어 픽셀 값이 [-1, 1] 범위

In [ ]:
dataiter = iter(test_loader)
images, labels = next(dataiter)

for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(images[i].squeeze(), cmap='gray')
plt.show() 

In [ ]:
#Numpy 순서에 맞게 변환
def imshow_grid(img):
  img = img / 2 + 0.5
  npimg = img.numpy()
  plt.imshow(np.transpose(npimg, (1,2,0)))
  plt.show()

* 파이토치는 3차원 image data를 C,W,H 순서로 저장 -> Channel First
* Matplotlib을 이용하여 영상을 출력하려면 W,H,C로 변경해줘야함 -> Channel Last

In [ ]:
output = torchvision.utils.make_grid(images[:6])
#image 출력
imshow_grid(output)

output = torchvision.utils.make_grid(images[:6])
- 현재 배치(images)에서 첫 6개의 이미지를 슬라이싱해 하나의 이미지 그리드로 만듦.

In [ ]:
#CNN 모델 생성
model =nn.Sequential(
    nn.Conv2d(1,10,kernel_size=5,padding=1),
    nn.MaxPool2d(2,2),
    nn.ReLU(),
    
    nn.Conv2d(10,20,kernel_size=5,stride=1),
    nn.MaxPool2d(2,2),
    nn.ReLU(),
    
    nn.Flatten(),
    nn.Linear(4*4*20, 10)
)

model =nn.Sequential(...): nn.Sequential 컨테이너를 사용해 CNN 모델을 순차적으로 정의.

nn.Conv2d(1,10,kernel_size=5,padding=1): 첫 번째 합성곱 계층.
    (in_channels=1): 입력 채널 수 (MNIST는 흑백이므로 1).
    (out_channels=10): 출력 채널 수 (생성할 특징 맵의 수).
    (kernel_size=5): 5*5 크기의 필터(커널)을 사용.
    (padding=1) 입력 이미지의 가장자리에 1픽셀의 패딩을 추가하여 출력 크기 감소를 줄임.

nn.MaxPool2d(2,2): 첫 번째 맥스 풀링 계층. 2x2 크기의 윈도우를 사용하여 각 윈도우 내의 최댓값을 선택하고, 스트라이드도 2로 설정하여 특징 맵의 크기를 가로세로 각각 절반으로 줄임.

nn.ReLU(): ReLU(Rectified Linear Unit) 활성화 함수를 적용.

nn.Conv2d(10,20,kernel_size=5,padding=1): 두 번째 합성곱 계층.
    (in_channels=10): 이전 계층의 출력 채널 수.
    (out_channels=20): 출력 채널 수 (생성할 특징 맵의 수).
    (kernel_size=5): 5*5 크기의 필터(커널)을 사용.
    (stride=1) 필터를 한 번에 1픽셀씩 이동시.

nn.MaxPool2d(2,2): 두 번째 맥스 풀링 계층.

nn.ReLU(): ReLU(Rectified Linear Unit) 활성화 함수를 적용.

nn.Flatten(): 다차원 특징 맵을 1차원 벡터로 평탄화.

nn.Linear(4*4*20, 10): 완전 연결(선형) 계층.

    (in_features=4*4*20)
- 입력 특징의 수. 이 값은 이전 합성곱 및 풀링 계층을 거치면서 계산된 특징 맵의 크기(4x4)와 채널 수(20)를 곱한 값입니다.

    (out_features=10)
- 출력 특징의 수 (MNIST는 0부터 9까지 10개의 클래스).

In [ ]:
torch.manual_seed(1)
model = model.to(device)
model

torch.manual_seed(1): 난수 시드를 1로 고정하여 가중치 초기화 등의 과정에서 재현성을 확보.

model = model.to(device): 정의된 model을 device 변수에 설정된 장치(CPU 또는 GPU)로 옮김.

model: 모델의 구조를 출력. 각 계층의 상세 정보(입출력 채널, 커널 크기, 스트라이드, 패딩 등)를 보여줌.

In [ ]:
#모델 학습
learning_rate = 0.001
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

learning_rate = 0.001: 학습률(learning rate)을 0.001로 설정. 학습률은 모델 파라미터를 업데이트하는 정도를 결정.

loss_function = nn.CrossEntropyLoss(): 손실 함수로 교차 엔트로피 손실(Cross-Entropy Loss)을 사용. 이 함수는 다중 클래스 분류 문제에 일반적으로 사용되며, 내부적으로 Softmax 함수를 포함.

optimizer = optim.Adam(model.parameters(), lr=learning_rate): 최적화 알고리즘으로 Adam(Adaptive Moment Estimation)을 사용.

    model.parameters(): 모델의 학습 가능한 모든 파라미터(가중치, 편향 등)를 최적화 대상으
    로 전달.
    
    lr=learning_rate: 최적화 알고리즘에 사용할 학습률을 지정.



In [ ]:
#epoch당 손실함수 출력
n_epochs = 10
for epoch in range(n_epochs):
    avg_loss = 0.0   

    for images, labels in train_loader:             
        images=images.to(device)
        labels=labels.to(device)
        outputs = model(images)
        loss = loss_function(outputs, labels)
        optimizer.zero_grad()  
        loss.backward()
        optimizer.step()         
        
        avg_loss += loss    
    avg_loss= avg_loss / len(train_loader)  
    print('Epoch : {:4d}/{:2d}, Loss = {:.5f}'.format( epoch+1, n_epochs, avg_loss))



n_epochs = 10: 전체 학습 데이터셋을 10번 반복하여 학습하도록 에폭 수를 설정.

for epoch in range(n_epochs):: 에폭 수만큼 반복하는 외부 루프.

    avg_loss = 0.0: 현재 에폭의 평균 손실을 저장할 변수를 0.0으로 초기화.
    
    for images, labels in train_loader:
    - 학습 데이터 로더(train_loader)에서 배치(batch) 단위로 이미지와 해당 레이블을 가져오는 내부 루프.

        images=images.to(device): 현재 배치의 이미지 데이터를 device(CPU 또는 GPU)로 옮김.
        
        labels=labels.to(device): 현재 배치의 레이블 데이터를 device로 옮김.
        
        outputs = model(images): 모델에 현재 배치의 이미지를 입력으로 넣어 예측값(outputs)을 얻음(순전파).
        
        loss = loss_function(outputs, labels): 모델의 예측값과 실제 레이블을 사용하여 손실(loss)을 계산.
        
        optimizer.zero_grad(): 이전 배치의 기울기(gradient) 값을 초기화. PyTorch는 기울기를 누적하므로, 각 배치마다 새로 계산하기 전에 초기화해야 함.
        
        loss.backward(): 계산된 손실에 대한 기울기를 역전파(backpropagation) 알고리즘을 통해 계산.
        
        optimizer.step(): 계산된 기울기를 사용하여 모델의 학습 가능한 파라미터들을 업데이트.
        
        avg_loss += loss: 현재 배치의 손실을 avg_loss에 누적. (주의: loss는 텐서이므로, 스칼라 값을 더하려면 loss.item()을 사용하는 것이 일반적. 여기서는 텐서 자체를 더하고 나중에 평균을 낸다.)

avg_loss= avg_loss / len(train_loader): 에폭의 평균 손실을 계산. len(train_loader)는 총 배치 수.

print(...): 현재 에폭 번호와 해당 에폭의 평균 손실을 지정된 형식으로 출력.

In [ ]:
# Test model using test data
with torch.no_grad():
    images = test_dataset.data.view(-1, 1, 28, 28).float()
    labels = test_dataset.targets  
    images=images.to(device)
    labels=labels.to(device)
    outputs = model(images)  
    predicted = torch.argmax(outputs, dim=1) == labels   
    accuracy = predicted.float().mean()
    print('Test Data Accuracy = {:.5f}'.format( accuracy)) 

with torch.no_grad():: 이 블록 내에서는 기울기 계산을 수행하지 않도록 설정. 모델 평가 시에는 파라미터 업데이트가 필요 없으므로 메모리 사용량을 줄이고 계산 속도를 높일 수 있다.

    images = test_dataset.data.view(-1, 1, 28, 28).float(): 
    - 테스트 데이터셋의 모든 이미지(test_dataset.data)를 가져와 모델 입력 형태에 맞게 차원을 변경(.view(-1, 1, 28, 28))하고, 타입을 float으로 변환. -1은 해당 차원의 크기를 자동으로 계산하도록 하며, 여기서는 전체 이미지 수를 의미. MNIST 이미지는 원래 (10000, 28, 28) 형태인데, 채널 차원(1)을 추가하여 (10000, 1, 28, 28)로 만듦.
    
    labels = test_dataset.targets: 테스트 데이터셋의 모든 실제 레이블을 가져옴.
    
    images=images.to(device): 이미지 데이터를 device로 옮김.
    
    labels=labels.to(device): 레이블 데이터를 device로 옮김.
    
    outputs = model(images): 모델에 전체 테스트 이미지를 입력하여 예측값을 얻는다.
    
    predicted = torch.argmax(outputs, dim=1) == labels: 
    - 모델의 출력(outputs)에서 각 샘플에 대해 가장 높은 값을 갖는 클래스의 인덱스(즉, 예측된 클래스)를 torch.argmax(outputs, dim=1)로 찾음. dim=1은 클래스 점수/확률이 있는 차원을 따라 최댓값을 찾도록 지정. 이 예측된 클래스와 실제 레이블(labels)을 비교하여 각 샘플에 대해 예측이 맞았는지(True) 틀렸는지(False)를 나타내는 불리언(boolean) 텐서를 생성.
    
    accuracy = predicted.float().mean(): 불리언 텐서 predicted를 float 타입으로 변환. (True는 1.0, False는 0.0). 그런 다음 .mean() 함수를 사용하여 전체 예측 중 맞은 예측의 비율, 즉 정확도를 계산.
    
    print(...): 테스트 데이터에 대한 정확도를 지정된 형식으로 출력.

In [ ]:
#torch.argmax()사용 예시
torch.manual_seed(1)
a = torch.randn(5,3)
print(a)
torch.argmax(a, dim =1)

torch.manual_seed(1): 난수 시드를 1로 고정.

a = torch.randn(5,3): 평균 0, 표준편차 1의 정규 분포에서 샘플링된 값으로 채워진 5x3 크기의 텐서 a를 생성.

print(a): 생성된 텐서 a를 출력.

torch.argmax(a, dim =1): 텐서 a의 각 행(dim=1은 행을 따라 연산)에서 최댓값을 갖는 요소의 인덱스를 반환. 결과는 각 행별 최댓값 인덱스를 담은 1차원 텐서.

In [ ]:
with torch.no_grad():
    images = test_dataset.data.view(-1, 1, 28, 28).float()
    labels = test_dataset.targets    
    images=images.to(device)
    labels=labels.to(device)    
    outputs = model(images)  
    predicted = torch.max(outputs, dim=1)[1] ==labels #argmax대신 max를 이용하여 계산
    accuracy = predicted.float().mean()
    print('Test Data Accuracy = {:.5f}'.format( accuracy)) 

#  #Test model using test data 와 거의 동일한 모델 평가 코드
images, labels device에 로드 후 images를 모델에 입력.
predicted = torch.max(outputs, dim=1)[1] ==labels: #argmax대신 max를 이용하여 계산
- torch.max(outputs, dim=1) 함수는 지정된 차원(dim=1)을 따라 각 행의 최댓값과 해당 최댓값의 인덱스를 모두 반환하는 튜플 (values, indices)을 생성. 여기서 [1]을 사용하여 인덱스만 선택하고, 이를 실제 레이블과 비교. 이는 torch.argmax()와 동일한 결과.

In [ ]:
#과정 재정리
torch.manual_seed(1)
#Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=64,
                                           shuffle=False)
model =nn.Sequential(
    nn.Conv2d(1,10,kernel_size=5,padding=1),
    nn.MaxPool2d(2,2),
    nn.ReLU(),
    
    nn.Conv2d(10,20,kernel_size=5,stride=1),
    nn.MaxPool2d(2,2),
    nn.ReLU(),
    
    nn.Flatten(),
    nn.Linear(4*4*20, 10)
)

torch.manual_seed(1)
model = model.to(device)
model
learning_rate = 0.001
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr =learning_rate)

데이터 로딩 -> 모델 정의 -> 최적화 설정

[데이터 로딩]
torch.manual_seed(1)
- 이전과 같은 시드 사용.

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                            batch_size=64, 
                                            shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=64, 
                                           shuffle=False) 
                                           
- 변환이 수행된 학습데이터와 검증데이터를 불러오는 각각의 DataLoader 생성.
- batch_size=64: 한 번에 64개 샘플씩 가져옴.
- shuffle=True: epoch마다 데이터를 무작위로 섞는다.(학습 성능 향상에 도움)
- shuffle=False: 테스트 데이터는 섞지 않고 순서대로 평가.

[모델 정의]
model =nn.Sequential(...): nn.Sequential 컨테이너를 사용해 CNN 모델을 순차적으로 정의.

nn.Conv2d(1,10,kernel_size=5,padding=1): 첫 번째 합성곱 계층.
    (in_channels=1): 입력 채널 수 (MNIST는 흑백이므로 1).
    (out_channels=10): 출력 채널 수 (생성할 특징 맵의 수).
    (kernel_size=5): 5*5 크기의 필터(커널)을 사용.
    (padding=1) 입력 이미지의 가장자리에 1픽셀의 패딩을 추가하여 출력 크기 감소를 줄임.

nn.MaxPool2d(2,2): 첫 번째 맥스 풀링 계층. 2x2 크기의 윈도우를 사용하여 각 윈도우 내의 최댓값을 선택하고, 스트라이드도 2로 설정하여 특징 맵의 크기를 가로세로 각각 절반으로 줄임.

nn.ReLU(): ReLU(Rectified Linear Unit) 활성화 함수를 적용.

nn.Conv2d(10,20,kernel_size=5,padding=1): 두 번째 합성곱 계층.
    (in_channels=10): 이전 계층의 출력 채널 수.
    (out_channels=20): 출력 채널 수 (생성할 특징 맵의 수).
    (kernel_size=5): 5*5 크기의 필터(커널)을 사용.
    (stride=1) 필터를 한 번에 1픽셀씩 이동시.

nn.MaxPool2d(2,2): 두 번째 맥스 풀링 계층.

nn.ReLU(): ReLU(Rectified Linear Unit) 활성화 함수를 적용.

nn.Flatten(): 다차원 특징 맵을 1차원 벡터로 평탄화.

nn.Linear(4*4*20, 10): 완전 연결(선형) 계층.

    (in_features=4*4*20)
- 입력 특징의 수. 이 값은 이전 합성곱 및 풀링 계층을 거치면서 계산된 특징 맵의 크기(4x4)와 채널 수(20)를 곱한 값입니다.

    (out_features=10)
- 출력 특징의 수 (MNIST는 0부터 9까지 10개의 클래스).

torch.manual_seed(1): 난수 시드를 1로 고정하여 가중치 초기화 등의 과정에서 재현성을 확보.

model = model.to(device): 정의된 model을 device 변수에 설정된 장치(CPU 또는 GPU)로 옮김.

model: 모델의 구조를 출력. 각 계층의 상세 정보(입출력 채널, 커널 크기, 스트라이드, 패딩 등)를 보여줌.

[최적화 설정]

learning_rate = 0.001: 학습률(learning rate)을 0.001로 설정. 학습률은 모델 파라미터를 업데이트하는 정도를 결정.

loss_function = nn.CrossEntropyLoss(): 손실 함수로 교차 엔트로피 손실(Cross-Entropy Loss)을 사용. 이 함수는 다중 클래스 분류 문제에 일반적으로 사용되며, 내부적으로 Softmax 함수를 포함.

optimizer = optim.Adam(model.parameters(), lr=learning_rate): 최적화 알고리즘으로 Adam(Adaptive Moment Estimation)을 사용.

    model.parameters(): 모델의 학습 가능한 모든 파라미터(가중치, 편향 등)를 최적화 대상으
    로 전달.
    
    lr=learning_rate: 최적화 알고리즘에 사용할 학습률을 지정.



지금까지 내용 통합하여 재정리

In [ ]:
#손실함수 그래프 그리기
torch.manual_seed(10)
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                            batch_size=64, 
                                            shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=64, 
                                           shuffle=False) 
model =nn.Sequential(
    nn.Conv2d(1,10,kernel_size=5,padding=1),
    nn.MaxPool2d(2,2),
    nn.ReLU(),
    
    nn.Conv2d(10,20,kernel_size=5,stride=1),
    nn.MaxPool2d(2,2),
    nn.ReLU(),
    
    nn.Flatten(),
    nn.Linear(4*4*20, 10)
)      
    
torch.manual_seed(1)
model = model.to(device)  
model

learning_rate = 0.001
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



데이터 로딩 -> 모델 정의 -> 최적화 설정

[데이터 로딩]
torch.manual_seed()
- 이전과 다른 시드 사용.

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                            batch_size=64, 
                                            shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=64, 
                                           shuffle=False) 
                                           
- 변환이 수행된 학습데이터와 검증데이터를 불러오는 각각의 DataLoader 생성.
- batch_size=64: 한 번에 64개 샘플씩 가져옴.
- shuffle=True: epoch마다 데이터를 무작위로 섞는다.(학습 성능 향상에 도움)
- shuffle=False: 테스트 데이터는 섞지 않고 순서대로 평가.

[모델 정의]
model =nn.Sequential(...): nn.Sequential 컨테이너를 사용해 CNN 모델을 순차적으로 정의.

nn.Conv2d(1,10,kernel_size=5,padding=1): 첫 번째 합성곱 계층.
    (in_channels=1): 입력 채널 수 (MNIST는 흑백이므로 1).
    (out_channels=10): 출력 채널 수 (생성할 특징 맵의 수).
    (kernel_size=5): 5*5 크기의 필터(커널)을 사용.
    (padding=1) 입력 이미지의 가장자리에 1픽셀의 패딩을 추가하여 출력 크기 감소를 줄임.

nn.MaxPool2d(2,2): 첫 번째 맥스 풀링 계층. 2x2 크기의 윈도우를 사용하여 각 윈도우 내의 최댓값을 선택하고, 스트라이드도 2로 설정하여 특징 맵의 크기를 가로세로 각각 절반으로 줄임.

nn.ReLU(): ReLU(Rectified Linear Unit) 활성화 함수를 적용.

nn.Conv2d(10,20,kernel_size=5,padding=1): 두 번째 합성곱 계층.
    (in_channels=10): 이전 계층의 출력 채널 수.
    (out_channels=20): 출력 채널 수 (생성할 특징 맵의 수).
    (kernel_size=5): 5*5 크기의 필터(커널)을 사용.
    (stride=1) 필터를 한 번에 1픽셀씩 이동시.

nn.MaxPool2d(2,2): 두 번째 맥스 풀링 계층.

nn.ReLU(): ReLU(Rectified Linear Unit) 활성화 함수를 적용.

nn.Flatten(): 다차원 특징 맵을 1차원 벡터로 평탄화.

nn.Linear(4*4*20, 10): 완전 연결(선형) 계층.

    (in_features=4*4*20)
- 입력 특징의 수. 이 값은 이전 합성곱 및 풀링 계층을 거치면서 계산된 특징 맵의 크기(4x4)와 채널 수(20)를 곱한 값입니다.

    (out_features=10)
- 출력 특징의 수 (MNIST는 0부터 9까지 10개의 클래스).

torch.manual_seed(1): 난수 시드를 1로 고정하여 가중치 초기화 등의 과정에서 재현성을 확보.

model = model.to(device): 정의된 model을 device 변수에 설정된 장치(CPU 또는 GPU)로 옮김.

model: 모델의 구조를 출력. 각 계층의 상세 정보(입출력 채널, 커널 크기, 스트라이드, 패딩 등)를 보여줌.

[최적화 설정]

learning_rate = 0.001: 학습률(learning rate)을 0.001로 설정. 학습률은 모델 파라미터를 업데이트하는 정도를 결정.

loss_function = nn.CrossEntropyLoss(): 손실 함수로 교차 엔트로피 손실(Cross-Entropy Loss)을 사용. 이 함수는 다중 클래스 분류 문제에 일반적으로 사용되며, 내부적으로 Softmax 함수를 포함.

optimizer = optim.Adam(model.parameters(), lr=learning_rate): 최적화 알고리즘으로 Adam(Adaptive Moment Estimation)을 사용.

    model.parameters(): 모델의 학습 가능한 모든 파라미터(가중치, 편향 등)를 최적화 대상으
    로 전달.
    
    lr=learning_rate: 최적화 알고리즘에 사용할 학습률을 지정.



In [ ]:
n_epochs = 10
train_losses=[]
test_losses=[]
train_accuracy=[]
test_accuracy=[]
for epoch in range(n_epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    for images, labels in train_loader: 
        images=images.to(device)
        labels=labels.to(device)
        outputs = model(images)
        loss = loss_function(outputs, labels)
        optimizer.zero_grad()  
        loss.backward()
        optimizer.step()         
               
        epoch_loss += loss.item()/len(train_loader)
        acc = ((outputs.argmax(dim=1) == labels).float().mean())
        epoch_accuracy += acc.item()/len(train_loader)
    train_losses.append(epoch_loss)  
    train_accuracy.append(epoch_accuracy)  
#    print('Epoch : {}, train loss : {:.5f}, train accuracy : {:.5f}'.format(epoch+1, epoch_loss, epoch_accuracy))
    
    with torch.no_grad():
        epoch_test_accuracy=0
        epoch_test_loss =0
        for images, labels in test_loader:          
            images=images.to(device)
            labels=labels.to(device)
            test_output = model(images)
            test_loss = loss_function(test_output,labels)
            
            epoch_test_loss += test_loss.item()/ len(test_loader)
            acc = ((test_output.argmax(dim=1) == labels).float().mean())
            epoch_test_accuracy += acc.item()/ len(test_loader)
        test_losses.append(epoch_test_loss) 
        test_accuracy.append(epoch_test_accuracy)  
#        print('Epoch : {}, test loss : {:.5f}, test accuracy : {:.5f}'.format(epoch+1, epoch_test_loss, epoch_test_accuracy))

1. 변수 초기화

이 셀은 모델을 10 에폭 동안 학습시키면서, 각 에폭이 끝날 때마다 학습 데이터셋과 테스트 데이터셋에 대한 평균 손실 및 정확도를 계산하여 각각의 리스트(train_losses, test_losses, train_accuracy, test_accuracy)에 저장.

2. 에폭 루프: 전체 학습 n_epochs만큼 반복.

- epoch_loss, epoch_accuracy: 한 에폭 동안의 누적 손실과 정확도를 저장할 변수


3. 학습 단계:
   
for images, labels in train_loader: 학습 데이터 전체를 배치 단위로 반복.

images, labels = images.to(device), labels.to(device): 데이터를 GPU 또는 CPU로 이동.

outputs = model(images): 모델에 이미지를 입력해 예측 결과(로짓)를 얻는다.

loss = loss_function(outputs, labels): 예측 결과와 실제 정답 레이블을 비교하여 손실값을 계산.

optimizer.zero_grad(): 이전 배치에서 계산된 기울기(gradient)를 초기화.

loss.backward(): 손실을 모델 파라미터에 대해 역전파(backpropagation)하여 기울기를 계산.

optimizer.step(): 계산된 기울기를 이용해 모델 파라미터를 업데이트.

epoch_loss += loss.item()/len(train_loader): 현재 배치의 손실값을 전체 배치 개수로 나눠 누적(평균 손실 계산).

acc = ((outputs.argmax(dim=1) == labels).float().mean()): 각 배치에서 예측값과 정답이 일치하는 비율(정확도)을 계산.

epoch_accuracy += acc.item()/len(train_loader): 배치별 정확도를 전체 배치 개수로 나눠 누적(평균 정확도 계산).

- loss.item()과 acc.item(): 텐서에서 스칼라 값을 추출하여 누적 합계에 사용합니다. 이렇게 해야 메모리 누수를 방지하고 정확한 평균값을 계산할 수 있다.

train_losses.append(epoch_loss), train_accuracy.append(epoch_accuracy): 한 에폭이 끝나면 평균 손실과 정확도를 리스트에 저장.



4. 평가 단계
with torch.no_grad(): 평가 단계에서는 기울기 계산이 필요 없으므로 메모리와 연산을 절약.

for images, labels in test_loader: 테스트 데이터를 배치 단위로 반복.

images, labels = images.to(device), labels.to(device): 데이터를 device로 이동.

test_output = model(images): 모델에 입력해 예측 결과를 얻는다.

test_loss = loss_function(test_output,labels): 예측값과 정답을 비교해 손실을 계산.

epoch_test_loss += test_loss.item()/ len(test_loader): 배치 손실을 전체 배치 개수로 나눠 누적(평균 손실).

acc = ((test_output.argmax(dim=1) == labels).float().mean()): 배치 정확도 계산.

epoch_test_accuracy += acc.item()/ len(test_loader): 배치 정확도를 전체 배치 개수로 나눠 누적(평균 정확도).

test_losses.append(epoch_test_loss), test_accuracy.append(epoch_test_accuracy): 한 에폭이 끝나면 평균 테스트 손실과 정확도를 리스트에 저장.

In [ ]:
#plot the loss function
plt.title("Train and Test Loss")
plt.plot(range(n_epochs),train_losses, label="train")
plt.plot(range(n_epochs),test_losses, label="test")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.title("Train and Test Loss"): 그래프의 제목을 "Train and Test Loss"로 설정.

plt.plot(range(n_epochs),train_losses, label="train"): x축을 에폭 번호(0부터 9), y축을 학습 손실(train_losses 리스트의 값)로 하여 학습 손실 곡선을 그림. label="train"은 범례에 표시될 이름.

plt.plot(range(n_epochs),test_losses, label="test"): x축을 에폭 번호, y축을 테스트 손실(test_losses 리스트의 값)로 하여 테스트 손실 곡선을 그림. label="test"는 범례에 표시될 이름.

plt.xlabel('Epoch'): x축의 레이블을 'Epoch'로 설정.

plt.ylabel('Loss'): y축의 레이블을 'Loss'로 설정.

plt.legend(): 그래프에 범례를 표시.

plt.show(): 생성된 그래프를 화면에 출력.

In [ ]:
#plot the accuracy function
plt.title("Train and Test accuracy")
plt.plot(range(n_epochs),train_accuracy, label="train")
plt.plot(range(n_epochs),test_accuracy, label="test")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.title("Train and Test accuracy"): 그래프의 제목을 "Train and Test accuracy"로 설정.

plt.plot(range(n_epochs),train_accuracy, label="train"): x축을 에폭 번호, y축을 학습 정확도(train_accuracy 리스트의 값)로 하여 학습 정확도 곡선을 그림.

plt.plot(range(n_epochs),test_accuracy, label="test"): x축을 에폭 번호, y축을 테스트 정확도(test_accuracy 리스트의 값)로 하여 테스트 정확도 곡선을 그림.

plt.xlabel('Epoch'): x축의 레이블을 'Epoch'로 설정.

plt.ylabel('Accuracy'): y축의 레이블을 'Accuracy'로 설정. (제공된 노트북 파일에서는 이 부분이 'Loss'로 되어 있으나, 문맥상 'Accuracy'가 맞습니다.)

plt.legend(): 그래프에 범례를 표시.

plt.show(): 생성된 그래프를 화면에 출력.

예제 8.2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.utils as utils
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print('Device:', device)

라이브러리 임포트 및 장치 설정

In [ ]:
train_dataset = torchvision.datasets.MNIST(root='MNIST_data',
                          train=True, 
                          transform=transforms.ToTensor(),
                          download=False)

test_dataset = torchvision.datasets.MNIST(root='MNIST_data',
                         train=False, 
                         transform=transforms.ToTensor(),
                         download=False)

 MNIST 데이터셋 로드 (ToTensor 변환만 적용)
 download=False로 설정되어 있으므로, 이전에 다운로드된 로컬 데이터를 사용합니다. 만약 데이터가 없다면 오류 발생

In [ ]:
# Check 
print(train_dataset.data.shape)
print(train_dataset.targets.shape)
print(test_dataset.data.shape)
print(test_dataset.targets.shape)

train_dataset.data.shape: 학습 데이터셋의 이미지 데이터 텐서의 모양을 출력 
(예: torch.Size([60000, 28, 28])).

train_dataset.targets.shape: 학습 데이터셋의 레이블 텐서의 모양을 출력 
(예: torch.Size([60000])).

test_dataset.data.shape: 테스트 데이터셋의 이미지 데이터 텐서의 모양을 출력.

test_dataset.targets.shape: 테스트 데이터셋의 레이블 텐서의 모양을 출력.

In [ ]:
torch.manual_seed(1)
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                            batch_size=64, 
                                            shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=64, 
                                           shuffle=False) 

MNIST 데이터셋을 위한 학습 및 테스트 데이터 로더를 생성.

이전 Data Loader 셀과 동일한 설정.

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1) 
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) 
        self.fc1 = nn.Linear(7 * 7 * 64, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 7 * 7 * 64)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

__init__(self): 모델 계층 초기화.

    self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1): 첫 번째 합성곱 계층. 입력 채널 1 (흑백), 출력 채널 32, 3x3 커널, 스트라이드 1, 패딩 1.
    
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2): 첫 번째 맥스 풀링 계층.
    
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1): 두 번째 합성곱 계층. 입력 채널 32, 출력 채널 64.
    
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2): 두 번째 맥스 풀링 계층.
    
    self.fc1 = nn.Linear(7 * 7 * 64, 128): 첫 번째 완전 연결 계층. 입력 특징 수는 7x7x64=3136 (28x28 이미지가 두 번의 2x2 풀링을 거치면 7x7이 되고, 채널 수는 64). 출력 특징 수는 128. 
    
    self.fc2 = nn.Linear(128, 10): 두 번째 완전 연결 계층(출력 계층). 입력 특징 수 128, 출력 특징 수 10 (0~9 숫자 클래스).

forward(self, x): 모델의 순전파(forward pass) 과정 정의.

    x = self.pool1(F.relu(self.conv1(x))): 입력 x는 conv1 -> ReLU 활성화 -> pool1을 순차적으로 통과.
    
    x = self.pool2(F.relu(self.conv2(x))): 이전 결과는 conv2 -> ReLU 활성화 -> pool2를 순차적으로 통과.
    
    x = x.view(-1, 7 * 7 * 64): 다차원 특징 맵을 완전 연결 계층에 입력하기 위해 1차원 벡터로 평탄화. -1은 배치 크기를 유지하도록 자동으로 계산.
    
    x = F.relu(self.fc1(x)): fc1 -> ReLU 활성화를 통과.
    
    x = self.fc2(x): fc2(출력 계층)를 통과. 일반적으로 분류 문제에서는 이 출력에 Softmax 함수를 적용하지만, nn.CrossEntropyLoss를 사용하면 내부적으로 Softmax가 처리되므로 여기서는 생략.
    
    return x: 모델의 최종 출력을 반환.

In [ ]:
torch.manual_seed(1)
model = CNN().to(device)  
model

torch.manual_seed(1): 난수 시드를 고정.

model = CNN().to(device): 위에서 정의한 CNN 클래스의 인스턴스(모델 객체) 생성, device로 옮김.

model: 모델의 구조를 출력.

In [ ]:
learning_rate = 0.001
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

학습률, 손실 함수, 최적화 도구를 설정

In [ ]:
n_epochs = 10
for epoch in range(n_epochs):
    avg_loss = 0.0   

    for images, labels in train_loader:             
        images=images.to(device)
        labels=labels.to(device)
        outputs = model(images)
        loss = loss_function(outputs, labels)
        optimizer.zero_grad()  
        loss.backward()
        optimizer.step()         
        
        avg_loss += loss    
    avg_loss= avg_loss / len(train_loader)  
    print('Epoch : {:4d}/{:2d}, Loss = {:.5f}'.format( epoch+1, n_epochs, avg_loss))

n_epochs = 10: 전체 학습 데이터셋을 10번 반복하여 학습하도록 에폭 수를 설정.

for epoch in range(n_epochs):: 에폭 수만큼 반복하는 외부 루프.

avg_loss = 0.0: 현재 에폭의 평균 손실을 저장할 변수를 0.0으로 초기화.

for images, labels in train_loader:
- 학습 데이터 로더(train_loader)에서 배치(batch) 단위로 이미지와 해당 레이블을 가져오는 내부 루프.

    images=images.to(device): 현재 배치의 이미지 데이터를 device(CPU 또는 GPU)로 옮김.
    
    labels=labels.to(device): 현재 배치의 레이블 데이터를 device로 옮김.
    
    outputs = model(images): 모델에 현재 배치의 이미지를 입력으로 넣어 예측값(outputs)을 얻음(순전파).
    
    loss = loss_function(outputs, labels): 모델의 예측값과 실제 레이블을 사용하여 손실(loss)을 계산.
    
    optimizer.zero_grad(): 이전 배치의 기울기(gradient) 값을 초기화. PyTorch는 기울기를 누적하므로, 각 배치마다 새로 계산하기 전에 초기화해야 함.
    
    loss.backward(): 계산된 손실에 대한 기울기를 역전파(backpropagation) 알고리즘을 통해 계산.
    
    optimizer.step(): 계산된 기울기를 사용하여 모델의 학습 가능한 파라미터들을 업데이트.
    
    avg_loss += loss: 현재 배치의 손실을 avg_loss에 누적. (주의: loss는 텐서이므로, 스칼라 값을 더하려면 loss.item()을 사용하는 것이 일반적. 여기서는 텐서 자체를 더하고 나중에 평균을 낸다.)
avg_loss= avg_loss / len(train_loader): 에폭의 평균 손실을 계산. len(train_loader)는 총 배치 수.

print(...): 현재 에폭 번호와 해당 에폭의 평균 손실을 지정된 형식으로 출력.

In [ ]:
# Test model using test data
with torch.no_grad():
    images = test_dataset.data.view(-1, 1, 28, 28).float()
    labels = test_dataset.targets    
    images=images.to(device)
    labels=labels.to(device)    
    outputs = model(images)  
    predicted = torch.max(outputs, dim=1)[1] ==labels
    accuracy = predicted.float().mean()
    print('Test Data Accuracy = {:.5f}'.format( accuracy)) 

#  #Test model using test data 와 거의 동일한 모델 평가 코드
images, labels device에 로드 후 images를 모델에 입력.
predicted = torch.max(outputs, dim=1)[1] ==labels: #argmax대신 max를 이용하여 계산
- torch.max(outputs, dim=1) 함수는 지정된 차원(dim=1)을 따라 각 행의 최댓값과 해당 최댓값의 인덱스를 모두 반환하는 튜플 (values, indices)을 생성. 여기서 [1]을 사용하여 인덱스만 선택하고, 이를 실제 레이블과 비교. 이는 torch.argmax()와 동일한 결과.

In [ ]:
#손실함수 그래프 그리기
torch.manual_seed(1)
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                            batch_size=64, 
                                            shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=64, 
                                           shuffle=False) 

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1) 
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) 
        self.fc1 = nn.Linear(7 * 7 * 64, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 7 * 7 * 64)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

torch.manual_seed(1)
model = CNN().to(device)  
model

learning_rate = 0.001
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

데이터 로딩 -> 모델 정의 -> 최적화 설정

[데이터 로딩]
torch.manual_seed()
- 이전과 다른 시드 사용.

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                            batch_size=64, 
                                            shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=64, 
                                           shuffle=False) 
                                           
- 변환이 수행된 학습데이터와 검증데이터를 불러오는 각각의 DataLoader 생성.
- batch_size=64: 한 번에 64개 샘플씩 가져옴.
- shuffle=True: epoch마다 데이터를 무작위로 섞는다.(학습 성능 향상에 도움)
- shuffle=False: 테스트 데이터는 섞지 않고 순서대로 평가.

[모델 정의]
__init__(self): 모델 계층 초기화.

self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1): 첫 번째 합성곱 계층. 입력 채널 1 (흑백), 출력 채널 32, 3x3 커널, 스트라이드 1, 패딩 1.
    
    self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2): 첫 번째 맥스 풀링 계층.
    
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1): 두 번째 합성곱 계층. 입력 채널 32, 출력 채널 64.
    
    self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2): 두 번째 맥스 풀링 계층.
    
    self.fc1 = nn.Linear(7 * 7 * 64, 128): 첫 번째 완전 연결 계층. 입력 특징 수는 7x7x64=3136 (28x28 이미지가 두 번의 2x2 풀링을 거치면 7x7이 되고, 채널 수는 64). 출력 특징 수는 128. 
    
    self.fc2 = nn.Linear(128, 10): 두 번째 완전 연결 계층(출력 계층). 입력 특징 수 128, 출력 특징 수 10 (0~9 숫자 클래스).

forward(self, x): 모델의 순전파(forward pass) 과정 정의.

    x = self.pool1(F.relu(self.conv1(x))): 입력 x는 conv1 -> ReLU 활성화 -> pool1을 순차적으로 통과.
    
    x = self.pool2(F.relu(self.conv2(x))): 이전 결과는 conv2 -> ReLU 활성화 -> pool2를 순차적으로 통과.
    
    x = x.view(-1, 7 * 7 * 64): 다차원 특징 맵을 완전 연결 계층에 입력하기 위해 1차원 벡터로 평탄화. -1은 배치 크기를 유지하도록 자동으로 계산.
    
    x = F.relu(self.fc1(x)): fc1 -> ReLU 활성화를 통과.
    
    x = self.fc2(x): fc2(출력 계층)를 통과. 일반적으로 분류 문제에서는 이 출력에 Softmax 함수를 적용하지만, nn.CrossEntropyLoss를 사용하면 내부적으로 Softmax가 처리되므로 여기서는 생략.
    
    return x: 모델의 최종 출력을 반환.

[최적화 설정]

learning_rate = 0.001: 학습률(learning rate)을 0.001로 설정. 학습률은 모델 파라미터를 업데이트하는 정도를 결정.

loss_function = nn.CrossEntropyLoss(): 손실 함수로 교차 엔트로피 손실(Cross-Entropy Loss)을 사용. 이 함수는 다중 클래스 분류 문제에 일반적으로 사용되며, 내부적으로 Softmax 함수를 포함.

optimizer = optim.Adam(model.parameters(), lr=learning_rate): 최적화 알고리즘으로 Adam(Adaptive Moment Estimation)을 사용.

    model.parameters(): 모델의 학습 가능한 모든 파라미터(가중치, 편향 등)를 최적화 대상으
    로 전달.
    
    lr=learning_rate: 최적화 알고리즘에 사용할 학습률을 지정.



In [ ]:
n_epochs = 10
train_losses=[]
test_losses=[]
train_accuracy=[]
test_accuracy=[]
for epoch in range(n_epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    for images, labels in train_loader: 
        images=images.to(device)
        labels=labels.to(device)
        outputs = model(images)
        loss = loss_function(outputs, labels)
        optimizer.zero_grad()  
        loss.backward()
        optimizer.step()         
               
        epoch_loss += loss.item()/len(train_loader)
        acc = ((outputs.argmax(dim=1) == labels).float().mean())
        epoch_accuracy += acc.item()/len(train_loader)
    train_losses.append(epoch_loss)  
    train_accuracy.append(epoch_accuracy)  
    print('Epoch : {}, train loss : {:.5f}, train accuracy : {:.5f}'.format(epoch+1, epoch_loss, epoch_accuracy))
    
    with torch.no_grad():
        epoch_test_accuracy=0
        epoch_test_loss =0
        for images, labels in test_loader:          
            images=images.to(device)
            labels=labels.to(device)
            test_output = model(images)
            test_loss = loss_function(test_output,labels)
            
            epoch_test_loss += test_loss.item()/ len(test_loader)
            acc = ((test_output.argmax(dim=1) == labels).float().mean())
            epoch_test_accuracy += acc.item()/ len(test_loader)
        test_losses.append(epoch_test_loss) 
        test_accuracy.append(epoch_test_accuracy)  
        print('Epoch : {}, test loss : {:.5f}, test accuracy : {:.5f}'.format(epoch+1, epoch_test_loss, epoch_test_accuracy))

1. 변수 초기화

이 셀은 모델을 10 에폭 동안 학습시키면서, 각 에폭이 끝날 때마다 학습 데이터셋과 테스트 데이터셋에 대한 평균 손실 및 정확도를 계산하여 각각의 리스트(train_losses, test_losses, train_accuracy, test_accuracy)에 저장.

2. 에폭 루프: 전체 학습 n_epochs만큼 반복.

- epoch_loss, epoch_accuracy: 한 에폭 동안의 누적 손실과 정확도를 저장할 변수


3. 학습 단계:
   
for images, labels in train_loader: 학습 데이터 전체를 배치 단위로 반복.

images, labels = images.to(device), labels.to(device): 데이터를 GPU 또는 CPU로 이동.

outputs = model(images): 모델에 이미지를 입력해 예측 결과(로짓)를 얻는다.

loss = loss_function(outputs, labels): 예측 결과와 실제 정답 레이블을 비교하여 손실값을 계산.

optimizer.zero_grad(): 이전 배치에서 계산된 기울기(gradient)를 초기화.

loss.backward(): 손실을 모델 파라미터에 대해 역전파(backpropagation)하여 기울기를 계산.

optimizer.step(): 계산된 기울기를 이용해 모델 파라미터를 업데이트.

epoch_loss += loss.item()/len(train_loader): 현재 배치의 손실값을 전체 배치 개수로 나눠 누적(평균 손실 계산).

acc = ((outputs.argmax(dim=1) == labels).float().mean()): 각 배치에서 예측값과 정답이 일치하는 비율(정확도)을 계산.

epoch_accuracy += acc.item()/len(train_loader): 배치별 정확도를 전체 배치 개수로 나눠 누적(평균 정확도 계산).

- loss.item()과 acc.item(): 텐서에서 스칼라 값을 추출하여 누적 합계에 사용합니다. 이렇게 해야 메모리 누수를 방지하고 정확한 평균값을 계산할 수 있다.

train_losses.append(epoch_loss), train_accuracy.append(epoch_accuracy): 한 에폭이 끝나면 평균 손실과 정확도를 리스트에 저장.

+) 각 에폭의 학습 및 테스트 손실/정확도를 print문을 통해 바로 출력.



4. 평가 단계
with torch.no_grad(): 평가 단계에서는 기울기 계산이 필요 없으므로 메모리와 연산을 절약.

for images, labels in test_loader: 테스트 데이터를 배치 단위로 반복.

images, labels = images.to(device), labels.to(device): 데이터를 device로 이동.

test_output = model(images): 모델에 입력해 예측 결과를 얻는다.

test_loss = loss_function(test_output,labels): 예측값과 정답을 비교해 손실을 계산.

epoch_test_loss += test_loss.item()/ len(test_loader): 배치 손실을 전체 배치 개수로 나눠 누적(평균 손실).

acc = ((test_output.argmax(dim=1) == labels).float().mean()): 배치 정확도 계산.

epoch_test_accuracy += acc.item()/ len(test_loader): 배치 정확도를 전체 배치 개수로 나눠 누적(평균 정확도).

test_losses.append(epoch_test_loss), test_accuracy.append(epoch_test_accuracy): 한 에폭이 끝나면 평균 테스트 손실과 정확도를 리스트에 저장.

+) 각 에폭의 학습 및 테스트 손실/정확도를 print문을 통해 바로 출력.

In [ ]:
#plot the loss function
plt.title("Train and Test Loss")
plt.plot(range(n_epochs),train_losses, label="train")
plt.plot(range(n_epochs),test_losses, label="test")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.title("Train and Test Loss"): 그래프의 제목을 "Train and Test Loss"로 설정.

plt.plot(range(n_epochs),train_losses, label="train"): x축을 에폭 번호(0부터 9), y축을 학습 손실(train_losses 리스트의 값)로 하여 학습 손실 곡선을 그림. label="train"은 범례에 표시될 이름.

plt.plot(range(n_epochs),test_losses, label="test"): x축을 에폭 번호, y축을 테스트 손실(test_losses 리스트의 값)로 하여 테스트 손실 곡선을 그림. label="test"는 범례에 표시될 이름.

plt.xlabel('Epoch'): x축의 레이블을 'Epoch'로 설정.

plt.ylabel('Loss'): y축의 레이블을 'Loss'로 설정.

plt.legend(): 그래프에 범례를 표시.

plt.show(): 생성된 그래프를 화면에 출력.

In [ ]:
#plot the accuracy function
plt.title("Train and Test accuracy")
plt.plot(range(n_epochs),train_accuracy, label="train")
plt.plot(range(n_epochs),test_accuracy, label="test")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.title("Train and Test accuracy"): 그래프의 제목을 "Train and Test accuracy"로 설정.

plt.plot(range(n_epochs),train_accuracy, label="train"): x축을 에폭 번호, y축을 학습 정확도(train_accuracy 리스트의 값)로 하여 학습 정확도 곡선을 그림.

plt.plot(range(n_epochs),test_accuracy, label="test"): x축을 에폭 번호, y축을 테스트 정확도(test_accuracy 리스트의 값)로 하여 테스트 정확도 곡선을 그림.

plt.xlabel('Epoch'): x축의 레이블을 'Epoch'로 설정.

plt.ylabel('Accuracy'): y축의 레이블을 'Accuracy'로 설정. 

plt.legend(): 그래프에 범례를 표시.

plt.show(): 생성된 그래프를 화면에 출력.

개미-벌 데이터 영상 분류

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from torchvision.datasets import ImageFolder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print('Device:', device)

Device: cpu


개미-벌 분류를 위한 라이브러리 임포트 및 장치 설정
from torchvision.datasets import ImageFolder: 이미지 파일이 클래스별로 정리된 디렉토리에서 데이터셋을 쉽게 로드할 수 있는 ImageFolder 클래스를 명시적으로 임포트.

In [3]:
data_transform = transforms.Compose([transforms.Resize((224,224))
                                       ,transforms.ToTensor()
                                       ,transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

이미지 데이터에 적용할 일련의 변환을 transforms.Compose를 사용하여 정의.

transforms.Resize((224,224)): 입력 이미지의 크기를 224x224 픽셀로 조절. 이는 많은 사전 훈련된 CNN 모델(예: ResNet, VGG)의 표준 입력 크기.

transforms.ToTensor(): PIL 이미지나 NumPy 배열을 PyTorch 텐서로 변환하고, 픽셀 값의 범위를 [0, 1]로 정규화.

transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]): 이미지의 각 채널(RGB)을 주어진 평균(mean)과 표준편차(std)를 사용하여 정규화합니다. 이 값들은 ImageNet 데이터셋의 통계치로, ImageNet으로 사전 훈련된 모델을 사용할 때 일반적으로 사용됩니다.

In [6]:
from google.colab import drive
drive.mount('/content/gdrive/')

ModuleNotFoundError: No module named 'google.colab'

Google Colaboratory 환경에서 사용자의 Google Drive를 /content/gdrive/ 경로에 마운트

In [ ]:
cd /content/gdrive/MyDrive

Colab의 현재 작업 디렉토리를 마운트된 Google Drive 내의 'MyDrive' 디렉토리로 변경

In [ ]:
train_dataset = ImageFolder('./hymenoptera_data/train/',transform =data_transform)
valid_dataset = ImageFolder('./hymenoptera_data/val/',transform =data_transform)

ImageFolder 클래스를 사용하여 개미와 벌 이미지 데이터셋을 로드.
    train_dataset = ImageFolder('./hymenoptera_data/train/', transform=data_transform): ./hymenoptera_data/train/ 경로에 있는 학습용 이미지 데이터셋을 로드. ImageFolder는 이 경로 아래에 있는 하위 디렉토리(예: 'ants', 'bees')를 클래스 레이블로 자동 인식. data_transform에 정의된 변환이 각 이미지에 적용.
    
    valid_dataset = ImageFolder('./hymenoptera_data/val/', transform=data_transform): ./hymenoptera_data/val/ 경로에 있는 검증용(validation) 이미지 데이터셋을 로드하며, 동일한 변환을 적용.


In [ ]:
print(len(train_dataset))
print(len(valid_dataset))
print(train_dataset[0][0].size())
print(valid_dataset[0][0].size())

print(len(train_dataset)): 학습 데이터셋의 총 이미지 수를 출력.

print(len(valid_dataset)): 검증 데이터셋의 총 이미지 수를 출력.

print(train_dataset[0][0].size()): 학습 데이터셋의 첫 번째 이미지 텐서의 크기를 출력. Resize((224,224))와 ToTensor() 변환으로 인해 (3, 224, 224) 형태 (채널, 높이, 너비)가 된다.

print(valid_dataset[0][0].size()): 검증 데이터셋의 첫 번째 이미지 텐서의 크기를 출력.

In [ ]:
class_names = train_dataset.classes
class_names_to_idx = train_dataset.class_to_idx
print(class_names)
print(class_names_to_idx)

class_names = train_dataset.classes: ImageFolder로 로드된 학습 데이터셋의 클래스 이름들을 리스트 형태로 가져옴 (예: ['ants', 'bees']).

class_names_to_idx = train_dataset.class_to_idx: 클래스 이름과 해당 클래스의 정수 인덱스를 매핑하는 딕셔너리를 가져옴 (예: {'ants': 0, 'bees': 1}).

print(class_names): 클래스 이름 리스트를 출력.

print(class_names_to_idx): 클래스 이름-인덱스 매핑 딕셔셔너리를 출력.

In [ ]:
train_dataset[0]

학습 데이터셋(train_dataset)의 첫 번째 샘플을 출력. 이는 (변환된 이미지 텐서, 해당 이미지의 레이블 인덱스) 형태의 튜플.

In [ ]:
train_dataset[0][0]

학습 데이터셋의 첫 번째 샘플에서 이미지 텐서(train_dataset[0][0])의 내용을 출력. Normalize 변환으로 인해 픽셀 값들이 특정 범위로 정규화되어 있다.

In [ ]:
torch.manual_seed(1)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4,num_workers=0,shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=4,num_workers=0)

개미-벌 이미지 데이터셋을 위한 학습 및 검증 데이터 로더를 생성.
    batch_size=4: 배치 크기를 4로 설정합니다.
    
    num_workers=0: 데이터 로딩에 사용할 서브프로세스의 수입니다. 0은 메인 프로세스에서 데이터를 로드함을 의미합니다.
    
    shuffle=True (train_loader): 학습 데이터는 에폭마다 섞습니다.
    
    shuffle=False (valid_loader, 기본값): 검증 데이터는 섞지 않습니다.

In [ ]:
print(len(train_loader)) #244/4
print(len(valid_loader)) #153/4

print(len(train_loader)): 학습 데이터 로더가 한 에폭당 생성하는 배치의 수를 출력. 주석 #244/4는 (총 학습 샘플 수 / 배치 크기)를 의미.

print(len(valid_loader)): 검증 데이터 로더가 생성하는 배치의 수를 출력. 주석 #153/4는 (총 검증 샘플 수 / 배치 크기)를 의미. (153/4 = 38.25이므로, 39개의 배치가 생성될 것이며 마지막 배치는 더 작을 수 있다.)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    """Imshow for Tensor."""
#    print(img.shape)
    img = img.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean
    img = np.clip(img, 0, 1)
#    print("max: {}, min: {}".format(np.max(img), np.min(img)))
    plt.imshow(img)
#    print(img.shape)

imshow(train_dataset[0][0])

이미지 시각화를 위한 imshow 함수를 다시 정의. 이번에는 컬러 이미지(3채널)를 처리하고, 셀 59에서 정의한 Normalize 변환을 되돌리는(역정규화) 과정을 포함.

    img = img.numpy().transpose((1, 2, 0)): PyTorch 텐서(C,H,W)를 NumPy 배열(H,W,C)로 변환.
    
    mean = np.array(...), std = np.array(...): 정규화 시 사용했던 평균과 표준편차 값을 정의.
    
    img = std * img + mean: 역정규화 연산 (input * std) + mean을 수행하여 픽셀 값을 원래 범위에 가깝게 복원.
    
    img = np.clip(img, 0, 1): 부동소수점 연산으로 인해 값이 [0, 1] 범위를 약간 벗어날 수 있으므로, 이 범위로 값을 제한(클리핑).

imshow(train_dataset[0][0]): 학습 데이터셋의 첫 번째 이미지를 이 함수를 사용하여 화면에 표시.

In [ ]:
from torchvision import transforms, utils
# Get a batch of training data
images, labels = next(iter(train_loader))
print(images.shape)
print(labels.shape)
# Make a grid from batch
img_grid = utils.make_grid(images)
imshow(img_grid)

1. 라이브러리 임포트
torchvision 패키지에서 데이터 전처리(transform)와 유틸리티 함수(utils)를 불러옴.
utils에는 이미지 그리드 생성 함수인 make_grid가 포함.

2. 배치 데이터 한 묶음 가져오기
train_loader는 DataLoader 객체로, 학습 데이터셋을 배치 단위로 반환.
iter(train_loader)로 이터레이터를 만들고, next()로 첫 번째 배치(이미지와 레이블)를 받아옴.


3. 배치 텐서 크기 확인
이미지 텐서와 레이블 텐서의 shape을 출력.

images의 shape은 [batch_size, channels, height, width] -> 4장의 224*224 RGB 이미지.
labels는 각 이미지의 정답 클래스 인덱스가 담긴 1차원 텐서.

4. 이미지 그리드 생성
make_grid 함수는 여러 이미지를 한 장의 그리드 이미지로 합쳐주기.

입력은 4차원 텐서(배치)여야 하며, 기본적으로 한 행에 8개씩 배치(nrow=8, 이미지가 8개 미만이면 한 행에 모두 표시).

반환값은 [channels, height, width] 형태의 3차원 텐서로, 합쳐진 그리드 이미지.

5. 그리드 이미지 시각화
imshow 함수는 텐서 이미지를 matplotlib로 시각화.

imshow 내부동작을 통해 numpy 배열화 후 역정규화 실행.


In [ ]:
#CNN 모델 생성
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size=(5, 5), stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size=(5, 5), stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=(3, 3), padding=1)
        self.fc1 = nn.Linear(in_features= 64 * 6 * 6, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=50)
        self.fc3 = nn.Linear(in_features=50, out_features=2)
        
        
    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2)
        
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2)
        
        X = F.relu(self.conv3(X))
        X = F.max_pool2d(X, 2)
        
#         print(X.shape)
        X = X.view(X.shape[0], -1)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        return X


#CNN 모델 생성: 주석으로, 개미-벌 이미지 분류를 위한 새로운 CNN 모델을 정의하는 부분임을 명시.

class CNN(nn.Module):: nn.Module을 상속받는 CNN 클래스를 정의.

__init__(self): 모델의 계층들을 초기화.

    self.conv1 = nn.Conv2d(in_channels = 3, ...): 첫 번째 합성곱 계층. 입력 채널이 3(컬러 이미지)으로 변경. 출력 채널 16, 5x5 커널, 스트라이드 2, 패딩 1.
    
    self.conv2 = nn.Conv2d(in_channels = 16, ...): 두 번째 합성곱 계층. 입력 채널 16, 출력 채널 32.
    
    self.conv3 = nn.Conv2d(in_channels = 32, ...): 세 번째 합성곱 계층. 입력 채널 32, 출력 채널 64, 3x3 커널.
    
    self.fc1 = nn.Linear(in_features= 64 * 6 * 6, ...): 첫 번째 완전 연결 계층. 입력 특징 수는 이전 합성곱 및 풀링 계층을 거친 후의 특징 맵 크기(6x6)와 채널 수(64)를 곱한 값(2304). 출력 특징 수는 500.
    
    self.fc2 = nn.Linear(in_features=500, out_features=50): 두 번째 완전 연결 계층.
    
    self.fc3 = nn.Linear(in_features=50, out_features=2): 세 번째 완전 연결 계층 (출력 계층). 출력 특징 수가 2로, 개미(ant)와 벌(bee) 두 개의 클래스를 분류.

forward(self, X): 모델의 순전파 과정을 정의합.

    X = F.relu(self.conv1(X)), X = F.max_pool2d(X, 2): 각 합성곱 계층 뒤에는 ReLU 활성화 함수와 2x2 맥스 풀링이 적용.
    
    # print(X.shape): 주석 처리된 코드로, 평탄화 전 특징 맵의 크기를 확인하는 데 사용할 수 있다.
    
    X = X.view(X.shape[0], -1): 다차원 특징 맵을 완전 연결 계층에 입력하기 위해 1차원 벡터로 평탄화.
    
    X = F.relu(self.fc1(X)), X = F.relu(self.fc2(X)): 완전 연결 계층 뒤에도 ReLU 활성화 함수가 적용.
    
    X = self.fc3(X): 마지막 출력 계층. 일반적으로 분류 문제에서는 이 출력에 Softmax 함수를 적용하지만, nn.CrossEntropyLoss를 손실 함수로 사용하면 내부적으로 Softmax가 처리되므로 여기서는 별도의 활성화 함수를 적용하지 않는다.
    
    return X: 모델의 최종 예측값(각 클래스에 대한 점수)을 반환.

In [ ]:
torch.manual_seed(1)
model = CNN().to(device)  
model

torch.manual_seed(1): 난수 시드를 고정.

model = CNN().to(device): 위에서 정의한 CNN 클래스의 인스턴스(모델 객체)를 생성하고, device로 옮기기.

model: 모델의 구조를 출력.

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
loss_function = nn.CrossEntropyLoss()

최적화 알고리즘으로 SGD(Stochastic Gradient Descent)를 사용. 학습률은 0.001, 모멘텀(momentum) 값은 0.9로 설정. 모멘텀은 이전 기울기의 방향을 현재 업데이트에 반영하여 수렴 속도를 높이고 지역 최적점(local minima)을 벗어나는 데 도움을 줄 수 있다.
loss_function = nn.CrossEntropyLoss(): 손실 함수로 교차 엔트로피 손실을 사용.

In [ ]:
n_epochs = 20
train_losses=[]
valid_losses=[]
train_accuracy=[]
valid_accuracy=[]
for epoch in range(n_epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    for images, labels in train_loader:
        images=images.to(device)
        labels=labels.to(device)    
        outputs = model(images)
        loss = loss_function(outputs, labels)
        optimizer.zero_grad()  
        loss.backward()
        optimizer.step()         
               
        epoch_loss += loss.item()/len(train_loader)
        acc = ((outputs.argmax(dim=1) == labels).float().mean())
        epoch_accuracy += acc.item()/len(train_loader)
    train_losses.append(epoch_loss)  
    train_accuracy.append(epoch_accuracy)  
    print('Epoch : {}, train loss : {:.5f}, train accuracy : {:.5f}'.format(epoch+1, epoch_loss, epoch_accuracy))
    
    with torch.no_grad():
        epoch_valid_accuracy=0
        epoch_valid_loss =0
        for images, labels in valid_loader:          
            images=images.to(device)
            labels=labels.to(device)    
            valid_outputs = model(images)
            valid_loss = loss_function(valid_outputs, labels)
            
            epoch_valid_loss += valid_loss.item()/ len(valid_loader)
            acc = ((valid_outputs.argmax(dim=1) == labels).float().mean())
            epoch_valid_accuracy += acc.item()/ len(valid_loader)
        valid_losses.append(epoch_valid_loss) 
        valid_accuracy.append(epoch_valid_accuracy)  
        print('Epoch : {}, test loss : {:.5f}, test accuracy : {:.5f}'.format(epoch+1, epoch_test_loss, epoch_test_accuracy))

1. 변수 초기화

이 셀은 모델을 10 에폭 동안 학습시키면서, 각 에폭이 끝날 때마다 학습 데이터셋과 검증 데이터셋에 대한 평균 손실 및 정확도를 계산하여 각각의 리스트(train_losses, valid_losses, train_accuracy, valid_accuracy)에 저장.

2. 에폭 루프: 전체 학습 n_epochs만큼 반복.

- epoch_loss, epoch_accuracy: 한 에폭 동안의 누적 손실과 정확도를 저장할 변수


3. 학습 단계:
   
for images, labels in train_loader: 학습 데이터 전체를 배치 단위로 반복.

images, labels = images.to(device), labels.to(device): 데이터를 GPU 또는 CPU로 이동.

outputs = model(images): 모델에 이미지를 입력해 예측 결과(로짓)를 얻는다.

loss = loss_function(outputs, labels): 예측 결과와 실제 정답 레이블을 비교하여 손실값을 계산.

optimizer.zero_grad(): 이전 배치에서 계산된 기울기(gradient)를 초기화.

loss.backward(): 손실을 모델 파라미터에 대해 역전파(backpropagation)하여 기울기를 계산.

optimizer.step(): 계산된 기울기를 이용해 모델 파라미터를 업데이트.

epoch_loss += loss.item()/len(train_loader): 현재 배치의 손실값을 전체 배치 개수로 나눠 누적(평균 손실 계산).

acc = ((outputs.argmax(dim=1) == labels).float().mean()): 각 배치에서 예측값과 정답이 일치하는 비율(정확도)을 계산.

epoch_accuracy += acc.item()/len(train_loader): 배치별 정확도를 전체 배치 개수로 나눠 누적(평균 정확도 계산).

- loss.item()과 acc.item(): 텐서에서 스칼라 값을 추출하여 누적 합계에 사용합니다. 이렇게 해야 메모리 누수를 방지하고 정확한 평균값을 계산할 수 있다.

train_losses.append(epoch_loss), train_accuracy.append(epoch_accuracy): 한 에폭이 끝나면 평균 손실과 정확도를 리스트에 저장.

+) 각 에폭의 학습 및 테스트 손실/정확도를 print문을 통해 바로 출력.



4. 평가 단계
with torch.no_grad(): 평가 단계에서는 기울기 계산이 필요 없으므로 메모리와 연산을 절약.

for images, labels in valid_loader: 검증 데이터를 배치 단위로 반복.

images, labels = images.to(device), labels.to(device): 데이터를 device로 이동.

valid_output = model(images): 모델에 입력해 예측 결과를 얻는다.

valid_loss = loss_function(valid_output,labels): 예측값과 정답을 비교해 손실을 계산.

epoch_valid_loss += valid_loss.item()/ len(valid_loader): 배치 손실을 전체 배치 개수로 나눠 누적(평균 손실).

acc = ((valid_output.argmax(dim=1) == labels).float().mean()): 배치 정확도 계산.

epoch_valid_accuracy += acc.item()/ len(valid_loader): 배치 정확도를 전체 배치 개수로 나눠 누적(평균 정확도).

valid_losses.append(epoch_valid_loss), valid_accuracy.append(epoch_valid_accuracy): 한 에폭이 끝나면 평균 테스트 손실과 정확도를 리스트에 저장.

+) 각 에폭의 학습 및 테스트 손실/정확도를 print문을 통해 바로 출력.

In [ ]:
#plot the loss function
plt.title("Train and Valid Loss")
plt.plot(range(n_epochs),train_losses, label="train")
plt.plot(range(n_epochs),valid_losses, label="valid")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.title("Train and Valid Loss"): 그래프의 제목을 "Train and Valid Loss"로 설정.

plt.plot(range(n_epochs),train_losses, label="train"): x축을 에폭 번호(0부터 9), y축을 학습 손실(train_losses 리스트의 값)로 하여 학습 손실 곡선을 그림. label="train"은 범례에 표시될 이름.

plt.plot(range(n_epochs),valid_losses, label="valid"): x축을 에폭 번호, y축을 검증 손실(valid_losses 리스트의 값)로 하여 테스트 손실 곡선을 그림. label="test"는 범례에 표시될 이름.

plt.xlabel('Epoch'): x축의 레이블을 'Epoch'로 설정.

plt.ylabel('Loss'): y축의 레이블을 'Loss'로 설정.

plt.legend(): 그래프에 범례를 표시.

plt.show(): 생성된 그래프를 화면에 출력.

In [ ]:
#plot the accuracy function
plt.title("Train and Valid accuracy")
plt.plot(range(n_epochs),train_accuracy, label="train")
plt.plot(range(n_epochs),valid_accuracy, label="valid")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.title("Train and Valid accuracy"): 그래프의 제목을 "Train and Valid accuracy"로 설정.

plt.plot(range(n_epochs),train_accuracy, label="train"): x축을 에폭 번호, y축을 학습 정확도(train_accuracy 리스트의 값)로 하여 학습 정확도 곡선을 그림.

plt.plot(range(n_epochs),valid_accuracy, label="valid"): x축을 에폭 번호, y축을 검증 정확도(test_accuracy 리스트의 값)로 하여 테스트 정확도 곡선을 그림.

plt.xlabel('Epoch'): x축의 레이블을 'Epoch'로 설정.

plt.ylabel('Accuracy'): y축의 레이블을 'Accuracy'로 설정. 

plt.legend(): 그래프에 범례를 표시.

plt.show(): 생성된 그래프를 화면에 출력.